# LDA Clean
[Tutorial Here](http://dataskunkworks.com/2018/06/06/extracting-topics-from-11000-newsgroups-posts-with-python-gensim-and-lda/)

### OVERVIEW
    1. GET the data
    2. CLEAN the data with regex
    3. TOKENIZE the data
    4. REMOVE stopwords from the data

In [16]:
import pandas as pd
def get_data(url):
    df = pd.read_json(url)
    return df

def do_some_eda(df):
    print(df.target_names.unique())
    df.head()

import re
def clean_data(df):
    
    text_corpus = df.content.values.tolist()
    text_corpus = [re.sub('(<|</)([A-Z])\w+>', '', doc) for doc in text_corpus] #removing things between <>
    text_corpus = [re.sub('\s+', ' ', doc) for doc in text_corpus] #removing newline 
    text_corpus = [re.sub("\'", "", doc) for doc in text_corpus] #removing single quotes
    return text_corpus

import gensim
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
 
def doc_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

        
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
 
def remove_stopwords(text):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in text_corpus]
 
    
import spacy
nlp = spacy.load('en', disable=['parser', 'ner'] )
 
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    print(len(texts))
    texts_out = []
    for idx, sent in enumerate(texts):
        if (idx) % 500 == 0:
            print(str(idx) + ' documents lemmatised')
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
 
# Create Dictionary
import gensim.corpora as corpora
def get_model(data_lemmatized):
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    corpus = [id2word.doc2bow(text) for text in data_lemmatized]

    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=20, 
                                               per_word_topics=True)
    return corpus, lda_model


def format_topics_sentences(ldamodel, corpus, texts):
    # Array of top 10 topics
    top10array = []
 
    for row in range(ldamodel.num_topics):
        wp = ldamodel.show_topic(row)
        topic_keywords = ", ".join([word for word, prop in wp])
        top10array.append((row+1, topic_keywords))
 
    top10dict = dict(top10array)
 
    sent_topics_df = pd.DataFrame(pd.DataFrame([sorted(topic[0], key=lambda x: (x[1]), reverse=True) 
                                                for topic in ldamodel[corpus]])[0])
    sent_topics_df.columns=["Data"]
    sent_topics_df['Dominant_Topic'] = sent_topics_df.Data.apply(lambda x: x[0]+1)
    sent_topics_df['Perc_Contribution'] = sent_topics_df.Data.apply(lambda x: round(x[1],4))
    sent_topics_df['Topic_Keywords'] = sent_topics_df.Dominant_Topic.apply(lambda x: top10dict[x])
 
    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents.rename("Text")], axis=1)
    sent_topics_df = sent_topics_df[['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords', 'Text']]
    return(sent_topics_df)

In [17]:
# def do_the_thing():
# df = get_data('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')

import os
def get_data_from_files(path):
    directory = os.listdir(path)
    results = []
    for file in directory:
        f=open(path+file,  encoding = "ISO-8859-1")
        results.append(f.read())
        f.close()
    return results


# DATA SET 2
data_fd = get_data_from_files('110/110-f-d/')
data_fr = get_data_from_files('110/110-f-r/')
data_md = get_data_from_files('110/110-m-d/')
data_mr = get_data_from_files('110/110-m-r/')

female_data = data_fd + data_fr 
male_data = data_md + data_mr
dem_data = data_md + data_fd
rep_data = data_mr + data_fr

all_data = female_data + male_data

# DATA SET 2 -- SMALL
female_data_sm = data_fd[:10] + data_fr[:10] 
male_data_sm = data_md[:10] + data_mr[:10]
dem_data = data_md[:10] + data_fd[:10]
rep_data = data_mr[:10] + data_fr[:10]

all_data = female_data_sm + male_data_sm
df = pd.DataFrame({'content': all_data})
df.head()

content
0  <DOC>\n<DOCNO>Mrs. JONES of Ohio. (PERSONAL EX...
1  <DOC>\n<DOCNO>Ms. ROS-LEHTINEN. (TOM LANTOS AN...
2  <DOC>\n<DOCNO>Ms. WATERS. (PROVIDING FOR CONSI...
3  <DOC>\n<DOCNO>Mrs. DAVIS of California. (PROVI...
4  <DOC>\n<DOCNO>Mrs. NAPOLITANO. (PASSENGER RAIL...

In [19]:
# do_some_eda(df)
text_corpus = clean_data(df)
words = list(doc_to_words(text_corpus)) 

words = remove_stopwords(words)
# words
data_lemmatized = lemmatization(words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

corpus, lda_model = get_model(data_lemmatized)

from pprint import pprint
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
df_topic_sents_keywords = format_topics_sentences(lda_model, corpus, text_corpus)
df_topic_sents_keywords


40
0 documents lemmatised
[(0,
  '0.008*"bill" + 0.008*"house" + 0.007*"say" + 0.007*"speaker" + 0.006*"year" '
  '+ 0.006*"support" + 0.006*"time" + 0.006*"would" + 0.006*"work" + '
  '0.005*"go"'),
 (1,
  '0.009*"bill" + 0.009*"house" + 0.008*"go" + 0.007*"would" + 0.007*"year" + '
  '0.006*"time" + 0.006*"not" + 0.006*"american" + 0.006*"speaker" + '
  '0.006*"representative"'),
 (2,
  '0.008*"house" + 0.008*"speaker" + 0.007*"would" + 0.007*"time" + '
  '0.007*"support" + 0.006*"year" + 0.006*"bill" + 0.006*"go" + 0.006*"make" + '
  '0.006*"act"'),
 (3,
  '0.009*"go" + 0.009*"representative" + 0.008*"house" + 0.008*"bill" + '
  '0.008*"people" + 0.007*"american" + 0.007*"say" + 0.007*"speaker" + '
  '0.007*"make" + 0.006*"year"'),
 (4,
  '0.009*"speaker" + 0.008*"house" + 0.008*"go" + 0.007*"make" + 0.006*"time" '
  '+ 0.006*"bill" + 0.006*"people" + 0.006*"american" + 0.006*"say" + '
  '0.006*"not"'),
 (5,
  '0.010*"house" + 0.008*"go" + 0.007*"work" + 0.007*"american" + '
  '0.00

Dominant_Topic  Perc_Contribution  \
0                1             0.5471   
1                1             0.2777   
2               10             0.3945   
3                6             0.3046   
4               13             0.6177   
5                1             0.2588   
6                1             0.6765   
7                2             0.3375   
8                6             0.2586   
9                1             0.5075   
10               1             0.4394   
11               6             0.7472   
12               6             0.2564   
13               2             0.4394   
14              11             0.5507   
15              11             0.5633   
16              13             0.2226   
17              13             0.4348   
18               1             0.4321   
19               6             0.4760   
20              17             0.5430   
21              13             0.2873   
22               6             0.3873   
23              17             0.4443   
24              11             0.2772   
25               8             0.2865   
26              10             0.5393   
27               2             0.2703   
28               6             0.4751   
29              17             0.3969   
30              11             0.6178   
31               2             0.5347   
32               2             0.2713   
33              11             0.3862   
34               2             0.7438   
35              13             0.4265   
36              17             0.3548   
37              13             0.4737   
38               1             0.9203   
39              11             0.2845   

                                       Topic_Keywords  \
0   bill, house, say, speaker, year, support, time...   
1   bill, house, say, speaker, year, support, time...   
2   house, go, speaker, bill, american, work, make...   
3   house, go, work, american, speaker, not, state...   
4   house, bill, act, say, go, state, american, re...   
5   bill, house, say, speaker, year, support, time...   
6   bill, house, say, speaker, year, support, time...   
7   bill, house, go, would, year, time, not, ameri...   
8   house, go, work, american, speaker, not, state...   
9   bill, house, say, speaker, year, support, time...   
10  bill, house, say, speaker, year, support, time...   
11  house, go, work, american, speaker, not, state...   
12  house, go, work, american, speaker, not, state...   
13  bill, house, go, would, year, time, not, ameri...   
14  go, house, speaker, bill, say, not, american, ...   
15  go, house, speaker, bill, say, not, american, ...   
16  house, bill, act, say, go, state, american, re...   
17  house, bill, act, say, go, state, american, re...   
18  bill, house, say, speaker, year, support, time...   
19  house, go, work, american, speaker, not, state...   
20  bill, house, go, would, people, speaker, time,...   
21  house, bill, act, say, go, state, american, re...   
22  house, go, work, american, speaker, not, state...   
23  bill, house, go, would, people, speaker, time,...   
24  go, house, speaker, bill, say, not, american, ...   
25  house, time, go, speaker, would, make, year, r...   
26  house, go, speaker, bill, american, work, make...   
27  bill, house, go, would, year, time, not, ameri...   
28  house, go, work, american, speaker, not, state...   
29  bill, house, go, would, people, speaker, time,...   
30  go, house, speaker, bill, say, not, american, ...   
31  bill, house, go, would, year, time, not, ameri...   
32  bill, house, go, would, year, time, not, ameri...   
33  go, house, speaker, bill, say, not, american, ...   
34  bill, house, go, would, year, time, not, ameri...   
35  house, bill, act, say, go, state, american, re...   
36  bill, house, go, would, people, speaker, time,...   
37  house, bill, act, say, go, state, american, re...   
38  bill, house, say, speaker, year, support, time...   
39  go, house, speaker, bill, say, not, am

In [ ]:
# # Visualize the topics
# import pyLDAvis
# import pyLDAvis.gensim  
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis


In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()
 
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')
 
for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)
 
# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]
 
# Show
sent_topics_sorteddf_mallet.head()